# Reinforcement-based Search Tree Pruning
## Abraham Oliver, Brown County High School

In [1]:
# Dependencies
import random, sys
import numpy as np
from importlib import reload
from copy import copy

# Deepnotakto Project
sys.path.insert(0, '..')
import util, visual, trainer, train, agents.agent
import environment as env
import agents.random_agent as rp
import agents.Q as Q
import agents.human as human

## The Game
#### 3 x 3
**Player 1 Winning Strategy** Play in the center on the first move. Play a knight's move (from chess) from the opponent's move.

#### 4 x 4
** Player 2 Winning Strategy** Draw an imaginary line either horizontally or vertically, splitting the board in half. Play a knight's move   from the opponent's move on the side of the imaginary line that the opponent's move was played.

#### 5 x 5, 6 x 6, and 7x7
**Player 1 Winning Strategy** Not yet known

#### 8x8 and larger
**Winner Not Known**

In [2]:
BOARD_SIZE = 3
# Create a human player (that can be used for both players)
h = human.Human()
# Create a 3x3 game environment
e = env.Env(BOARD_SIZE)
# Play games between the humans on the 3x3
gui = visual.GameWithConfidences(e, h, h, -1, show_confidences = False)

SystemExit: 

C:\Users\abeol\Anaconda3\envs\Deep-Notakto\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Q-Learning Agents
#### Definitions
##### Markov Decision Process (MDP)
A markov decision process is a decision process that is defined by the tuple $(S, A, R_p(\cdot), \gamma)$ where $S$ is a state space (space of possible board positions), $A$ is an action space (space of possible actions for an agent to make), $R_p(s)$ is the immediate reward for some $s \in S$ and a given player, and $\gamma$ is the discount factor (the balance between future and immediate rewards). An time step of a deterministic MDP at time $t$ is $(s_t,a_t,r_t)$ where $r_t = R_p(s_t)$. An agent in an MDP is optimized in order to maximize the expected discounted reward from a given time-step $t$ until a terminal state at time-step $T$, $R_T=\mathbb{E}[\sum_{n=t}^{T} \gamma^{n-t} r_t]$.
##### Q-Learning
In this environment, there exists a function $Q^*: S \to A$ that produces the action $a$ that maximizes $R_T$ for a given state. Because it is often impossible to find the true $Q^*$, we approximate $Q^*$ with a funtion $Q_\pi: S \to A$ that produces an action based on a given policy $\pi$ (note that $Q=Q^*$ when $\pi=\pi^*$, the optimal policy). We define $Q$ by $Q_\pi(s)=\mathrm{argmax}_a\mathbb{E}[R_T|s, a, \pi]$.
##### Q-Agent
For a computer agent, Q is defined by a neural network that accepts a given board state and returns a probability distribution over the action space. After a game rollout is completed, we can train the neural network by calculating target Q-values using the Bellman Equation $Q_{\mathrm{target}}(s_t)=r_t+\gamma \mathrm{max}_{a'}Q(s',a')$.

In [4]:
# Create a Q-Agent
training_parameters = {"mode": "none", "learn_rate": .01, "rotate": True, "epochs": 1, "batch_size": 1, "replay_size": 10}
a1 = Q(layers = [9, 10, 9], gamma = .4, params = training_parameters, temp_func = 1)
# Create a 3x3 environment
e = env.Env(3)
# Create a random player
r = rp.RandomAgent(e)
# Play the Q-Agent against the random player
a1.deterministic = True
gui = visual.GameWithConfidences(e, a1, r, -1, show_confidences = True)

SystemExit: 

C:\Users\abeol\Anaconda3\envs\Deep-Notakto\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
# Load a Q-Agent
a2 = util.load_agent("agents/saves/p3.npz", Q)
a2.change_param("mode", "none")
a2.deterministic = True
# Play the Q-Agent against the random player
gui = visual.GameWithConfidences(e, a2, r, -1, show_confidences = True)

SystemExit: 

C:\Users\abeol\Anaconda3\envs\Deep-Notakto\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Training

In [ ]:
# Train the agent
a1.change_param("mode", "replay")
a1.deterministic = False
train.train_agent(e, a1, r, 10, 100)

In [ ]:
# Play the newly trained agent
a1.change_param("mode", "none")
a1.deterministic = True
gui = visual.GameWithConfidences(e, a1, r, -1, show_confidences = True)